In [1]:
from pystac_client import Client
import pystac
import stackstac
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.raster import RasterExtension

# Resolve ints stored as floats

Goal of this notebook is to test and resolve the integers being stored as floats in the MAAP VEDA pgSTAC database.

## 1. Determinte what extensions are being used to identify which schemas to check for integer fields which need to be updated.

What extensions are being used?
in veda data pipelines, we use rio_stac.create_stac_item to build stac metadata. 
https://github.com/MAAP-Project/veda-data-pipelines/blob/main/lambdas/build-stac/utils/stac.py#L56

I couldn't know what version of rio-stac is being used, but downloading the build-stac lambda docker image from our MAAP VEDA TEst stack in UAH returned 0.4.2

```
root@2b637b648fe7:/app# pip show rio-stac
Name: rio-stac
Version: 0.4.2
```

We are not using the [extensions](https://github.com/developmentseed/rio-stac/blob/0.4.2/rio_stac/stac.py#L211) argument that you can pass but we are using [`with_proj` and `with_raster`](https://github.com/developmentseed/rio-stac/blob/0.4.2/rio_stac/stac.py#L221-L222) arguments, which then use the proj and raster extensions.

https://stac-extensions.github.io/projection/v1.0.0/schema.json

* "type": "integer"
* proj:epsg and proj:shape

https://stac-extensions.github.io/raster/v1.1.0/schema.json

* "type": "integer"
* bands-> items -> properties -> bits_per_sample
    * Appears bits_per_sample is missing so we don't need to be concerned with it https://github.com/developmentseed/rio-stac/blob/0.4.2/rio_stac/stac.py#L136
* bands-> items -> properties -> histogram -> properties -> buckets

## 2. Determine which collections to test, fix and re-test for error

Iterate through each collection and find which ones have either the raster or proj extension

In [62]:
catalog_url = "https://stac.maap-project.org"

def has_projection_extension(item):
    return ProjectionExtension.has_extension(item)

def has_raster_extension(item):
    return RasterExtension.has_extension(item)

client = Client.open(catalog_url)

collections_with_extensions = []

for collection in client.get_all_collections():
    items = collection.get_items()
    try:
        first_item = next(iter(items), None)
    except Exception as e:
        print(e)
        print(collection.id)
        next

    if first_item:
        if has_projection_extension(first_item) or has_raster_extension(first_item):
            print(f"Collection '{collection.id}' - First item has projection and/or raster extension.")
            collections_with_extensions.append(collection.id)
        else:
            print(f"Collection '{collection.id}' - First item does NOT have the projection or raster extensions.")
    else:
        print(f"Collection '{collection.id}' - No items found.")

Collection 'GlobCover_09' - First item does NOT have the projection or raster extensions.
Collection 'AfriSAR_UAVSAR_Coreg_SLC' - First item does NOT have the projection or raster extensions.
Collection 'BIOSAR1' - First item does NOT have the projection or raster extensions.
Collection 'AfriSAR_AGB_Maps_1681' - First item has projection and/or raster extension.
Collection 'Landsat8_SurfaceReflectance' - First item does NOT have the projection or raster extensions.
Collection 'ABLVIS1B' - First item has projection and/or raster extension.
Collection 'AfriSAR_UAVSAR_Ungeocoded_Covariance' - First item does NOT have the projection or raster extensions.
Collection 'GlobCover_05_06' - First item does NOT have the projection or raster extensions.
Collection 'Global_PALSAR2_PALSAR_FNF' - First item does NOT have the projection or raster extensions.
Collection 'GEDI02_A' - First item does NOT have the projection or raster extensions.
Collection 'AFLVIS2' - First item does NOT have the project

In [63]:
collections_with_extensions

['AfriSAR_AGB_Maps_1681', 'ABLVIS1B', 'icesat2-boreal']

## 3. Test each collection fails those fields are integers

In [67]:
# for each collection, check the proj:epsg, proj:shape are floats and 
# raster:bands
client = Client.open(catalog_url)

for collection_id in collections_with_extensions:
    collection = client.get_collection(collection_id)
    items = collection.get_items()
    first_item = next(iter(items), None)
    print(collection_id)
    print(first_item.properties.get("proj:epsg", 'no epsg code'))  # 4326.0
    print(first_item.properties.get("proj:shape", 'no shape'))
    first_asset = first_item.assets.get('cog_default', '')
    if first_asset:
        print(first_asset.extra_fields['raster:bands'][0]['histogram']['buckets'])
    print(RasterExtension.has_extension(first_item)) 
    print('\n')

AfriSAR_AGB_Maps_1681
32732.0
[101.0, 105.0]
True


ABLVIS1B
no epsg code
[512.0, 512.0]
False


icesat2-boreal
no epsg code
[3000.0, 3000.0]
[681765.0, 228373.0, 63820.0, 21697.0, 9756.0, 4666.0, 2226.0, 760.0, 103.0, 15.0]
True




## 4. Fix the collection

In [57]:
import psycopg2

def connect_and_execute(database, user, password, host, port, sql):
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=database,
            user=user,
            password=password,
            host=host,
            port=port
        )
        cursor = conn.cursor()
        print(cursor.execute(sql))

        # Fetch results if the SQL statement is a SELECT query
        if sql.strip().lower().startswith('select'):
            results = cursor.fetchall()
            print("Results:")
            for row in results:
                print(row)

        conn.commit()
        cursor.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
    finally:
        if conn is not None:
            conn.close()

fix_epsg_sql = f"""
EXPLAIN ANALYZE
WITH collection_items AS (
    SELECT id
    FROM items
    WHERE collection = '{collection_id}'
    AND (content->'properties'->>'proj:shape')::text ~ '(\d+)\.0+\b'
)
UPDATE items
SET content = jsonb_set(
    content,
    '{{proj:shape}}',
    regexp_replace(
        (content->'properties'->>'proj:shape')::text,
        '(\d+)\.0+\b',
        '\1',
        'g'
    )::jsonb
)
FROM collection_items
WHERE items.id = collection_items.id;
"""

fix_shape_sql = f"""
UPDATE items
SET content = jsonb_set(
    content,
    '{{properties, proj:shape}}',
    (
        SELECT jsonb_agg(CAST(FLOOR(CAST(value AS NUMERIC)) AS INTEGER))
        FROM jsonb_array_elements_text(content #> '{{properties, proj:shape}}')
    )
)
WHERE collection = '{collection_id}';
"""

fix_bands_sql = f"""
CREATE OR REPLACE FUNCTION update_buckets_to_integers()
RETURNS VOID AS $$
DECLARE
    item_id TEXT;
    bands JSONB;
    updated_bands JSONB;
    band JSONB;
    index INTEGER := 0;
BEGIN
    FOR item_id, bands IN
        SELECT id, content #> '{{assets, cog_default, raster:bands}}' FROM items
        WHERE collection = '{collection_id}'
    LOOP
        updated_bands := '[]'::JSONB;
        
        FOR band IN SELECT * FROM jsonb_array_elements(bands)
        LOOP
            band := jsonb_set(
                band,
                '{{histogram, buckets}}',
                (SELECT jsonb_agg(CAST(FLOOR(value::TEXT::NUMERIC) AS INTEGER))
                 FROM jsonb_array_elements(band #> '{{histogram, buckets}}'))
            );
            updated_bands := updated_bands || band;
        END LOOP;
        
        UPDATE items
        SET content = jsonb_set(content, '{{assets, cog_default, raster:bands}}', updated_bands)
        WHERE id = item_id;
    END LOOP;
END;
$$ LANGUAGE plpgsql;

/* run the FUNCTION */
SELECT dashboard.update_buckets_to_integers();
"""

check_it_worked = f"""
WITH raster_band_elements AS (
    SELECT jsonb_array_elements(content #> '{{assets, cog_default, raster:bands}}') AS raster_bands
    FROM items
    WHERE collection = '{collection_id}'
),
histogram_elements AS (
    SELECT raster_bands #> '{{histogram, buckets}}' AS buckets
    FROM raster_band_elements
)
SELECT * FROM histogram_elements;
"""

# Connect to database
host = "veda-backend-test-postgres.coy9fgtxpaqn.us-west-2.rds.amazonaws.com"
database = "postgis"
user = "delta"
password = os.environ.get['DB_PWD']
port = 5432

# Run query
collection_id = 's1-rtc-seasonal-composite'
connect_and_execute(database, user, password, host, port, fix_epsg_sql)
connect_and_execute(database, user, password, host, port, fix_shape_sql)
connect_and_execute(database, user, password, host, port, fix_bands_sql)

None
None
None


## 5. Test it is fixed

Re-run code in part 3.